# Evaluation of $\pi$ calculation using DSMC

## Introduction

This notebook is used to evaluate a collection of experimental assays used to estimate $\pi$ with PRNGs and the QRNG.

The basis for the statistical tests introduced in this notebook are the PRNG and QRNG datasets consisting of multiple repetitions
of the following set-up:

- An experimental assay of
        - A varying number of batches: 2, 8, 16, 64, 128, 256
        - With a varying number of repetitions of the Pi estimation within each batch: 10, 20, 40, 80, 100, 150, 200
        - And a varying number of sample points to estimate Pi for each repetition: 100 000, 1 000 000

This notebook is used to process the raw data of _all_ experimental assays (**it is assumed that the same number of runs exists for PRNGs and the QRNG**), store the accumulated data and compare the results.

### Notation

In this document we try to adhere to the following notation:
- The true value of the constant is denoted $\pi$.
- An approximate value obtained by sampling the unit square is denoted $\tilde{\pi}_i = \frac{\text{points within unit circle}}{\text{total points}}$, where $i$ is the index of the sample value.
- An averaging operation is denoted by $\langle \cdot\rangle_N$, where $N$ indicates the number of values being averaged over.
- An expectation value and variance of a random variable $X$ are denoted as $\mathbb{E}[X],Var[X]$ and their empirical equivalents are $\mu(x) = \langle X \rangle_N, s_n^2(x)=\frac{1}{n-1}\sum_{i=1}^{N} (x_i - \mu(x))$

## Imports

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os, shutil, sys, re

In [ ]:
from scipy import stats

In [ ]:
figsize=(6,5)

## Raw data processing

### Preparations

First we define the assay parameters along with the directories for QRNG and PRNG data

In [ ]:
assayRootdirQRNG = "/programs/extension/QDice_tests/PiTesting/RUN/asymptoticAssay_buffon_QRNG_errorfixed_NETDEVICE/run"
assayRootdirPRNG = "/programs/extension/QDice_tests/PiTesting/RUN/asymptoticAssay_buffon_pPRNG_mrg5s/run"

In [ ]:
assayDirNameTemplateQRNG="{assayId:d}/{precision:s}"
assayDirNameTemplatePRNG="{assayId:d}/{precision:s}"

**Note**: For data generated using Snakemake workflows the parameters below are taken from the workflow's `configuration.yaml` and could, in principle, be parsed automatically.

In [ ]:
#parallel run parameters
Nbatches= [4, 8, 16, 32, 64, 128, 256]
Nrep= [10, 20, 40, 60, 80, 100, 120, 140, 160, 180, 200]
Nsamples=[int(i) for i in [10000, 100000] ]

NumAssays = 10

precision="double"

Change into the directory of the QRNG experiments

In [ ]:
assayDirQRNG = assayRootdirQRNG+'/'+assayDirNameTemplateQRNG.format(assayId=1, precision=precision)

In [ ]:
os.chdir(assayDirQRNG)

A sample name of the results, as codified in the `assay.sh` scripts is: `results_assay2__batchSize_8_Nrep_10_samples_1000000.txt`

Using the naming scheme we define a template for the result filename

In [ ]:
ResultFilenameTemplate="results_assay_{assayId:d}__batchSize_{batchSize:d}_Nrep_{Nrep:d}_samples_{Nsamples:d}.txt"

In [ ]:
!cat results_assay_1__batchSize_16_Nrep_10_samples_100000.txt

The above shows the contents of a given case. The result file contains two lines:
- The first line contains #repetitions, `<pi>`, `Var[pi]`, [list of #repetitions approximations]
- The second line contains #repetitions, `|pi-<pi>|`, `Var[|pi-<pi>|]`, [list of #repetitions abs. errors]

Using this information we may create a parser for the results. The function takes a file name and outputs a dictionary containing:    
- No. of repetitions $N$ - integer
- $\langle \tilde{\pi} \rangle_N$ - scalar
- $Var[\tilde{\pi}]$ - scalar
- $\langle |\tilde{\pi} - \pi| \rangle_N$ - scalar
- $Var[ | \tilde{\pi} - \pi |]$ - scalar
- $\tilde{\pi}_i$ - numpy array of samples
- $|\tilde{\pi}_i - \pi |$ - numpy array of samples

In [ ]:
def parseResults(filename:str):
    data = np.genfromtxt(filename, delimiter=',')
    data = data[:,:-1]
    results = {}
    results["Nrep"] = int(data[0,0])
    results['meanPi'] = data[0,1]
    results['varPi'] = data[0,2]
    results['meanAbsErr'] = data[1,1]
    results['varAbsErr'] = data[1,2]
    
    results['piSamples'] = np.copy(data[0,3:])
    results['absErrSamples'] = np.copy(data[1,3:])
    return results

This notebook is used to evaluate data obtained with code which includes the iteration over the batch into the C++ source. In this case the output is of size 2*`batchSize` and consists of pairs of lines structured as presented above.

In [ ]:
def parseBatchedResults(filename:str, batchSize:int):
    '''
    The routine is used to obtain results from DSMC calculations of 
    pi that were run in batched mode of the C++ code.
    In this case the output file contains ``batchSize`` pairs of results
    as defined above.
    '''
    data = np.genfromtxt(filename, delimiter=',')
    obtainedBatchSize = data.shape[0]//2
    assert obtainedBatchSize == batchSize, "Wrong batch size!"
    data = data[:,:-1]
    Nrep = int(data[0,0])
    means = np.zeros(batchSize)
    var = np.zeros(batchSize)
    errorMeans = np.zeros(batchSize)
    errorVariances = np.zeros(batchSize)
    samples = np.zeros((batchSize, Nrep)) 
    errorSamples = np.zeros((batchSize, Nrep)) 
    #copy data appropriately
    means = np.copy(data[0::2, 1])
    var = np.copy(data[0::2, 2])
    errorMeans = np.copy( data[1::2, 1] )
    errorVariances = np.copy( data[1::2, 2] )
    samples = np.copy( data[0::2, 3:] )
    errorSamples = np.copy( data[0::2, 3:] )
    return (means, var, errorMeans, errorVariances, samples, errorSamples)

### QRNG data collection

Prior to collecting and storing the data we note the following:

The varying number of batches prescribed in `Nbatches` is arbitrary and can be varied _a-posteriori_ since it influences only the calculation of the _mean of means_ and of the confidence interval.

Hence grouping of the data is performed using only the number of repetitions `N` and number of samples `S`.

In [ ]:
PointsPerRepAndSampleSize = np.sum( np.array(Nbatches) )
print(PointsPerRepAndSampleSize)

This defines how many datapoints each configuration $(N,S)$ will yield.

**This will likely change in the future to use either Pandas or Xarray for a more flexible data storage and ease of visualisation**

In [ ]:
means_batched1 = np.zeros( (NumAssays, len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )
variances_batched1 = np.zeros( (NumAssays, len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )

absErrorMeans_batched1 = np.zeros( (NumAssays, len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )
absErrorVariances_batched1 = np.zeros( (NumAssays, len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )

In [ ]:
#iterate over the assays
for assayId in range(1, NumAssays+1):
    assayDirQRNG = assayRootdirQRNG+'/'+assayDirNameTemplateQRNG.format(assayId=assayId, precision=precision)
    os.chdir(assayDirQRNG)
    
    batchOffsetIdx = 0;
    #iterate over the batch sizes
    for batchSize in Nbatches:
            #iterate over the number of repetitions
            for nrep, repIdx in zip(Nrep, range(len(Nrep) ) ):
                #terate over the number of samples
                for nSamples, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
                    filenameParameters= {'assayId':assayId,'batchSize':batchSize, 'Nrep':nrep, 'Nsamples':nSamples}
                    filename = ResultFilenameTemplate.format(**filenameParameters)
                    data = parseResults(filename)
                    batchMeans, batchVar, batchErrorMeans, batchErrorVariances, samples, sampleErrors = parseBatchedResults(filename, batchSize)
                    #distribute data to the arrays
                    means_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchMeans)
                    variances_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchVar)
                    
                    absErrorMeans_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchErrorMeans)
                    absErrorVariances_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchErrorVariances)
                    # end of the samples loop
                #end of the Nrep loop
            batchOffsetIdx += batchSize # increment the running index to store the data appropriately
    #end of the batch loop
    os.chdir(assayRootdirQRNG) #return to the common directory

Collection of the raw $\tilde{\pi}_i$ samples does not require a distinction between assays, numbers of repetitions or batches. Each $\tilde{\pi}_i$ is a sample in its own right and the distinction is only made by the number of sample points  on $[0,1]\times[0,1]$ used to obtain the $\tilde{\pi}_i$.

In [ ]:
# Storage for raw samples - distinguishable only by the number of points on [0,1]^2 used to obtain the estimates

rawSamples = []
rawSampleErrors = []

Collect sample data using Pandas

In [ ]:
#iterate over the assays
for nSamples, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
    sampleData = []
    sampleErrorData = []
    for assayId in range(1, NumAssays+1):
        assayDirQRNG = assayRootdirQRNG+'/'+assayDirNameTemplateQRNG.format(assayId=assayId, precision=precision)
        os.chdir(assayDirQRNG)
        #iterate over the number of batches
        for batchSize in Nbatches:
            #iterate over the number of repetitions
            for nrep, repIdx in zip(Nrep, range(len(Nrep) ) ):
                filenameParameters= {'assayId':assayId,'batchSize':batchSize, 'Nrep':nrep, 'Nsamples':nSamples}
                filename = ResultFilenameTemplate.format(**filenameParameters)
                rawData = pd.read_csv(filename, header=None)
                sampleData.append(rawData.iloc[::2,3:-1].values.flatten())
                sampleErrorData.append(rawData.iloc[1::2,3:-1].values.flatten())
            #end of Nrep loop
        #end of batch size loop
    os.chdir(assayRootdirQRNG)
    #end assay loop
    rawSamples.append( sampleData )
    rawSampleErrors.append( sampleErrorData)
    #end of the data collection

Due to the use of multiple assays the samples have to be rearranged

In [ ]:
indexedSamples = {}.fromkeys(Nsamples)
indexedAbsErrSamples = {}.fromkeys(Nsamples)

In [ ]:
for key, idx in zip(indexedSamples, range(len(Nsamples))):
    indexedSamples[key] = np.concatenate( rawSamples[idx] )
    indexedAbsErrSamples[key] = np.concatenate( rawSampleErrors[idx] )

In [ ]:
samplesDF = pd.DataFrame(indexedSamples)

In [ ]:
absErrSamplesDF = pd.DataFrame(indexedAbsErrSamples)

In [ ]:
assert samplesDF.shape[0] == np.sum(Nrep) * np.sum(Nbatches) * NumAssays, "Incorrect number of points collected!"

Store to CSV files

In [ ]:
samplesDF.to_csv("rawSamples_parallel_asymptotic_{precision:s}_QRNG.csv".format(precision=precision))
absErrSamplesDF.to_csv("rawAbsErrorSamples_parallel_asymptotic_{precision:s}_QRNG.csv".format(precision=precision))

### Simple visualisation

In [ ]:
Nbins = int( np.ceil(1 + np.log2( samplesDF.shape[0] )) )
samplesDF.hist(bins=Nbins)

### PRNG data collection and evaluation

In [ ]:
os.chdir(assayRootdirPRNG)

In [ ]:
prngmeans_batched1 = np.zeros( (NumAssays,len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )
prngvariances_batched1 = np.zeros( (NumAssays,len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )

prngabsErrorMeans_batched1 = np.zeros( (NumAssays,len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )
prngabsErrorVariances_batched1 = np.zeros( (NumAssays,len(Nrep), len(Nsamples), PointsPerRepAndSampleSize), dtype=float )

In [ ]:
#iterate over the assays
for assayId in range(1, NumAssays+1):
    assayDirPRNG = assayRootdirPRNG+'/'+assayDirNameTemplatePRNG.format(assayId=assayId, precision=precision)
    os.chdir(assayDirPRNG)
    
    batchOffsetIdx = 0;
    #iterate over the batch sizes
    for batchSize in Nbatches:
            #iterate over the number of repetitions
            for nrep, repIdx in zip(Nrep, range(len(Nrep) ) ):
                #terate over the number of samples
                for nSamples, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
                    filenameParameters= {'assayId':assayId,'batchSize':batchSize, 'Nrep':nrep, 'Nsamples':nSamples}
                    filename = ResultFilenameTemplate.format(**filenameParameters)
                    data = parseResults(filename)
                    batchMeans, batchVar, batchErrorMeans, batchErrorVariances, samples, sampleErrors = parseBatchedResults(filename, batchSize)
                    #distribute data to the arrays
                    prngmeans_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchMeans)
                    prngvariances_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchVar)
                    
                    prngabsErrorMeans_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchErrorMeans)
                    prngabsErrorVariances_batched1[assayId-1, repIdx, nSamplesIdx, batchOffsetIdx:batchOffsetIdx + batchSize] = np.copy(batchErrorVariances)
                    # end of the samples loop
                #end of the Nrep loop
            batchOffsetIdx += batchSize # increment the running index to store the data appropriately
    #end of the batch loop
    os.chdir(assayRootdirPRNG) #return to the common directory

In [ ]:
# Storage for raw samples - distinguishable only by the number of points on [0,1]^2 used to obtain the estimates
rawSamples = []
rawSampleErrors = []

Collect sample data using Pandas

In [ ]:
#iterate over the assays
for nSamples, nSamplesIdx in zip(Nsamples, range(len(Nsamples)) ):
    sampleData = []
    sampleErrorData = []
    for assayId in range(1, NumAssays+1):
        assayDirPRNG = assayRootdirPRNG+'/'+assayDirNameTemplatePRNG.format(assayId=assayId, precision=precision)
        os.chdir(assayDirPRNG)
        #iterate over the number of batches
        for batchSize in Nbatches:
            #iterate over the number of repetitions
            for nrep, repIdx in zip(Nrep, range(len(Nrep) ) ):
                filenameParameters= {'assayId':assayId,'batchSize':batchSize, 'Nrep':nrep, 'Nsamples':nSamples}
                filename = ResultFilenameTemplate.format(**filenameParameters)
                rawData = pd.read_csv(filename, header=None)
                sampleData.append(rawData.iloc[::2,3:-1].values.flatten())
                sampleErrorData.append(rawData.iloc[1::2,3:-1].values.flatten())
            #end of Nrep loop
        #end of batch size loop
    os.chdir(assayRootdirPRNG)
    #end assay loop
    rawSamples.append( sampleData )
    rawSampleErrors.append( sampleErrorData)
    #end of the data collection

In [ ]:
indexedSamples = {}.fromkeys(Nsamples)
indexedAbsErrSamples = {}.fromkeys(Nsamples)

In [ ]:
for key, idx in zip(indexedSamples, range(len(Nsamples))):
    indexedSamples[key] = np.concatenate( rawSamples[idx] )
    indexedAbsErrSamples[key] = np.concatenate( rawSampleErrors[idx] )

In [ ]:
samplesDF_prng = pd.DataFrame(indexedSamples)

In [ ]:
absErrSamplesDF_prng = pd.DataFrame(indexedAbsErrSamples)

In [ ]:
samplesDF_prng.to_csv("rawSamples_parallel_asymptotic_{precision:s}_pPRNG.csv".format(precision=precision))
absErrSamplesDF_prng.to_csv("rawAbsErrorSamples_parallel_asymptotic_{precision:s}_pPRNG.csv".format(precision=precision))

In [ ]:
os.chdir(assayRootdirPRNG)

-------------------

The preceding parts of the notebook are used to accumulate raw data into CSV files. These should then be processed by the `DSMCPIEvaluation_MulticaseMerged.ipynb` notebook.

The sections following below serve to provide a first glance at the distributions of the freshly processed data.

#### Combine the raw samples of QRNG and PRNG into a single data frame

In [ ]:
st1p = samplesDF_prng.T

st1q = samplesDF.T

In [ ]:
combinedSamples_prng = samplesDF_prng
combinedSamples_qrng = samplesDF

In [ ]:
combinedSamples_prng.hist(density=True, bins=Nbins)

In [ ]:
combinedSamples_qrng.hist(density=True, bins=Nbins)

### Simple visualisation of PRNG and QRNG data

Define confidence parameter $\alpha$ and the confidence interval for the mean-of-means.

In [ ]:
alpha=0.05
CIrange=1-alpha/2

In [ ]:
means_batched1.shape

In [ ]:
prngcase1_means = prngmeans_batched1[:,-1,-1,:].flatten()
prngcase1_variances = prngvariances_batched1[:,-1,-1,:].flatten()


qrngcase1_means = means_batched1[:,-1,-1,:].flatten()
qrngcase1_variances = variances_batched1[:,-1,-1,:].flatten()

In [ ]:
prngcase1_means.shape

In [ ]:
prngbatchMeans = np.zeros( len(Nbatches) )
prngbatchVariances = np.zeros_like(prngbatchMeans)
prngweightingFactors = np.zeros_like(prngbatchMeans)


qrngbatchMeans = np.zeros( len(Nbatches) )
qrngbatchVariances = np.zeros_like(qrngbatchMeans)
qrngweightingFactors = np.zeros_like(qrngbatchMeans)

In [ ]:
for i in range(len(Nbatches)):
    #PRNG
    prngbatchMeans[i] = np.average( prngcase1_means[:Nbatches[i]] )
    prngbatchVariances[i] = np.var( prngcase1_means[:Nbatches[i]], ddof=1 )
    prngweightingFactors[i] = stats.t.ppf(CIrange, Nbatches[i] - 1)
    #QRNG
    qrngbatchMeans[i] = np.average( qrngcase1_means[:Nbatches[i]] )
    qrngbatchVariances[i] = np.var( qrngcase1_means[:Nbatches[i]], ddof=1 )
    qrngweightingFactors[i] = stats.t.ppf(CIrange, Nbatches[i] - 1)

In [ ]:
prngerrorbarLimits = np.zeros_like(prngweightingFactors)
qrngerrorbarLimits = np.zeros_like(qrngweightingFactors)

In [ ]:
for i in range(len(Nbatches)):
    prngerrorbarLimits[i] = prngweightingFactors[i] * np.sqrt(prngbatchVariances[i] / Nbatches[i] )
    qrngerrorbarLimits[i] = qrngweightingFactors[i] * np.sqrt(qrngbatchVariances[i] / Nbatches[i] )

Plot a combined errorbar plot

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = plt.gca()

ax.errorbar( Nbatches, qrngbatchMeans, yerr=qrngerrorbarLimits, marker='o', capsize=5, capthick=4, lw=0,  label="QRNG" )
ax.errorbar( Nbatches, prngbatchMeans, yerr=prngerrorbarLimits, marker='x',capsize=5,capthick=4, lw=0, label="PRNG - MT19937" )
ax.axhline(np.pi, color='r', label="$\pi$")
ax.legend(loc='best')

ax.set_xlabel("Batch size", fontsize=14)
ax.set_xscale('log', base=2)
ax.set_ylabel("$\\langle \\tilde{\pi}\\rangle$", fontsize=14)

ax.set_title("DSMC $\pi$ approx. - $N_{samples}=10^6$")

ax.grid(which='both')
#fig.savefig("DSMC_QRNG_PRNG_CI_vs_ BatchSize_99percentCI.png")

### Visualisation of multiple confidence intervals

In [ ]:
qrngcase1_means.shape[0]

In [ ]:
NsamplesMultibatchVis = qrngcase1_means.shape[0]//4

In [ ]:
prngbatchMeans = np.zeros( 4 )
prngbatchVariances = np.zeros_like(prngbatchMeans)
prngweightingFactors = np.zeros_like(prngbatchMeans)

In [ ]:
qrngbatchMeans = np.zeros( 4 )
qrngbatchVariances = np.zeros_like(qrngbatchMeans)
qrngweightingFactors = np.zeros_like(qrngbatchMeans)

In [ ]:
for i in range(4):
    prngbatchMeans[i] = np.average( prngcase1_means[i*NsamplesMultibatchVis:(i+1)*NsamplesMultibatchVis] )
    prngbatchVariances[i] = np.var( prngcase1_means[i*NsamplesMultibatchVis:(i+1)*NsamplesMultibatchVis], ddof=1 )
    prngweightingFactors[i] = stats.t.ppf(CIrange, NsamplesMultibatchVis - 1)
    qrngbatchMeans[i] = np.average( qrngcase1_means[i*NsamplesMultibatchVis:(i+1)*NsamplesMultibatchVis] )
    qrngbatchVariances[i] = np.var( qrngcase1_means[i*NsamplesMultibatchVis:(i+1)*NsamplesMultibatchVis], ddof=1 )
    qrngweightingFactors[i] = stats.t.ppf(CIrange, NsamplesMultibatchVis - 1)    

In [ ]:
prngerrorbarLimits = np.zeros_like(prngweightingFactors)
qrngerrorbarLimits = np.zeros_like(qrngweightingFactors)

for i in range(4):
    prngerrorbarLimits[i] = prngweightingFactors[i] * np.sqrt(prngbatchVariances[i] / NsamplesMultibatchVis )
    qrngerrorbarLimits[i] = qrngweightingFactors[i] * np.sqrt(qrngbatchVariances[i] / NsamplesMultibatchVis )

In [ ]:
CIlocations= np.arange(1,4+1)

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = plt.gca()

ax.errorbar( CIlocations, qrngbatchMeans, yerr=qrngerrorbarLimits, marker='o', capsize=5, capthick=4, lw=0,  label="method 1" )
ax.errorbar( CIlocations+0.25, prngbatchMeans, yerr=prngerrorbarLimits, marker='o', capsize=5, capthick=4, lw=0,  label="method 2" )
ax.axhline(np.pi, color='r', label="True value")
ax.legend(loc='best', fontsize=14)

ax.set_xlabel("Batch number", fontsize=14)
ax.set_ylabel("$\\langle \\tilde{\pi}\\rangle$", fontsize=14)
ax.set_ylim([np.pi-1e-4, np.pi+1e-4])
ax.set_yticks([np.pi-1e-4, np.pi-1e-4/4, np.pi,np.pi+1e-4/4, np.pi+1e-4])
ax.set_xticks(CIlocations)
ax.set_yticklabels(["$\\pi-10^{-4}$", "$\\pi-1/4\cdot10^{-4}$", "$\\pi$","$\\pi+1/4\cdot10^{-4}$", "$\\pi+10^{-4}$"], fontsize=14)
ax.set_title("99% Confidence intervals of 104 measuerements", fontsize=14)

ax.grid(which='major')
#fig.savefig("DSMC_CIvariation.png", dpi=150)

#### Visualising confidence intervals with violin plots

In [ ]:
case1_means_split = np.split(qrngcase1_means, 4)
prngcase1_means_split = np.split(prngcase1_means, 4)

In [ ]:
fig = plt.figure(figsize=(10,10))

ax = plt.gca()

ax.violinplot(case1_means_split, positions=CIlocations,  showmedians=True)
ax.violinplot(prngcase1_means_split, positions=(CIlocations+0.25),  showmedians=True)
ax.errorbar( CIlocations, qrngbatchMeans, yerr=qrngerrorbarLimits, marker='o', capsize=5, capthick=4, lw=0,  label="QRNG" )
ax.errorbar( CIlocations+0.25, prngbatchMeans, yerr=prngerrorbarLimits, marker='o', capsize=5, capthick=4, lw=0,  label="PRNG" )
ax.axhline(np.pi, color='r', label="True value")
ax.legend(loc='best', fontsize=14)


ax.set_xlabel("Batch number", fontsize=14)
ax.set_ylabel("$\\langle \\tilde{\pi}\\rangle$", fontsize=14)
#ax.set_ylim([np.pi-1e-4, np.pi+1e-4])
#ax.set_yticks([np.pi-1e-4, np.pi-1e-4/4, np.pi,np.pi+1e-4/4, np.pi+1e-4])
ax.set_xticks(CIlocations)
#ax.set_yticklabels(["$\\pi-10^{-4}$", "$\\pi-1/4\cdot10^{-4}$", "$\\pi$","$\\pi+1/4\cdot10^{-4}$", "$\\pi+10^{-4}$"], fontsize=14)
ax.set_title("Distribution and 99% CI of 104 measuerements", fontsize=14)


ax.grid(which='major')
#fig.savefig("DSMC_CIvariationWithDist.png", dpi=150)